In [1]:
import pandas as pd
train_data = pd.read_csv("train.csv")
import h2o
h2o.init(nthreads = -1, max_mem_size = 8)


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 days 0 hours 25 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,3 months and 19 days !!!
H2O cluster name:,H2O_from_python_babs4JESUS_c6btew
H2O cluster total nodes:,1
H2O cluster free memory:,6.787 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [6]:
y_column = 'Survived'
x_columns = 'Sex'

In [26]:
clean_h2o_data = h2o.H2OFrame(train_data)    
clean_h2o_data['Sex'] = clean_h2o_data['Sex'].asfactor()
clean_h2o_data['Survived'] = clean_h2o_data['Survived'].asfactor()


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [27]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator


In [28]:
# Do a 10 fold cross validation as that is done typically.
rf_fit_cross_val = H2ORandomForestEstimator(model_id='rf_fit_cross_val', seed=1)
rf_fit_cross_val.train(x=x_columns, y=y_column, 
                       training_frame=clean_h2o_data)


drf Model Build progress: |███████████████████████████████████████████████| 100%


In [29]:
survived_set = clean_h2o_data['Survived']
survived_set['predictions'] = rf_fit_cross_val.predict(clean_h2o_data)['predict']

drf prediction progress: |████████████████████████████████████████████████| 100%


In [30]:
survived_set_df = survived_set.as_data_frame()

In [31]:
rf_fit_cross_val

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_fit_cross_val


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.16668250151251918
RMSE: 0.40826768364948895
LogLoss: 0.5150414885568484
Mean Per-Class Error: 0.37105742498322314
AUC: 0.757959714100065
Gini: 0.51591942820013
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7420382272629511: 


,0,1,Error,Rate
0,514.0,35.0,0.0638,(35.0/549.0)
1,232.0,110.0,0.6784,(232.0/342.0)
Total,746.0,145.0,0.2997,(267.0/891.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7420382,0.7103659,13.0
max f2,0.1889081,0.7569721,34.0
max f0point5,0.7420382,0.7290363,13.0
max accuracy,0.7420382,0.7867565,13.0
max precision,0.7420382,0.7586207,8.0
max recall,0.1889081,1.0,34.0
max specificity,0.7420382,0.9890710,0.0
max absolute_mcc,0.7420382,0.5433514,13.0
max min_per_class_accuracy,0.1889084,0.7163743,14.0
max mean_per_class_accuracy,0.7420382,0.7668728,13.0


Gains/Lift Table: Avg response rate: 38.38 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0224467,0.7420382,1.8236842,1.8236842,0.7,0.7,0.0409357,0.0409357,82.3684211,82.3684211
,2,0.0336700,0.7420382,1.8236842,1.8236842,0.7,0.7,0.0204678,0.0614035,82.3684211,82.3684211
,3,0.0493827,0.7420382,1.6748120,1.7763158,0.6428571,0.6818182,0.0263158,0.0877193,67.4812030,77.6315789
,4,0.0549944,0.7420382,2.6052632,1.8609023,1.0,0.7142857,0.0146199,0.1023392,160.5263158,86.0902256
,5,0.1234568,0.7420382,2.0073339,1.9421053,0.7704918,0.7454545,0.1374269,0.2397661,100.7333909,94.2105263
,6,0.1627385,0.7420382,2.0842105,1.9764065,0.8,0.7586207,0.0818713,0.3216374,108.4210526,97.6406534
,7,0.2312009,0.7420382,1.7937877,1.9223301,0.6885246,0.7378641,0.1228070,0.4444444,79.3787748,92.2330097
,8,0.3142536,0.7420382,1.7603129,1.8795113,0.6756757,0.7214286,0.1461988,0.5906433,76.0312945,87.9511278
,9,0.4242424,0.1889084,1.1431257,1.6885965,0.4387755,0.6481481,0.1257310,0.7163743,14.3125671,68.8596491
,10,0.5072952,0.1889084,0.3520626,1.4697834,0.1351351,0.5641593,0.0292398,0.7456140,-64.7937411,46.9783419



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-06-25 14:49:41,0.000 sec,0.0,nan,nan,nan,nan,nan
,2018-06-25 14:49:41,0.002 sec,1.0,0.4159935,0.5291761,0.7564076,1.8641107,0.2257053
,2018-06-25 14:49:41,0.003 sec,2.0,0.4133440,0.5242423,0.7554480,1.8752169,0.2215909
,2018-06-25 14:49:41,0.005 sec,3.0,0.4085014,0.5148531,0.7634202,1.8679245,0.2153846
,2018-06-25 14:49:41,0.006 sec,4.0,0.4108006,0.5194015,0.7611488,1.8537449,0.2180350
---,---,---,---,---,---,---,---,---
,2018-06-25 14:49:41,0.085 sec,46.0,0.4082677,0.5150415,0.7636000,2.0131579,0.2132435
,2018-06-25 14:49:41,0.087 sec,47.0,0.4082677,0.5150415,0.7585616,1.8036437,0.2132435
,2018-06-25 14:49:41,0.088 sec,48.0,0.4082677,0.5150415,0.7633257,1.8333333,0.2132435
,2018-06-25 14:49:41,0.090 sec,49.0,0.4082677,0.5150415,0.7648462,1.9539474,0.2132435



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Sex,3110.6569824,1.0,1.0


In [39]:
len(survived_set_df[(survived_set_df['Survived'] == 0) & (survived_set_df['predictions'] == 1)])

81